In [1]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
import numpy as np
import re
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [2]:
all_features = ["hashtags", "tweet_type", "Language", "tweet_timestamp", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager",\
                "like_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [1, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01")

# train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
# n_head = 90
# n_tail = 10
# train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
# val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()

In [9]:
train_1 = train.loc[:, train.columns != 'hashtags']

In [19]:
train_subset = train_1.head(10)

In [25]:
train_subset.to_dict(orient='records')

[{'tweet_type': 'TopLevel',
  'Language': '22C448FF81263D4BAF2A176145EE9EAD',
  'tweet_timestamp': 1581258144,
  'enaged_with_user_id': 'D7EFC6D19CFB09B9A1CE4C4B0E0091F6',
  'engaged_with_user_follower_count': 3517,
  'engaged_with_user_following_count': 590,
  'engaged_with_user_is_verified': False,
  'engaging_user_id': '0000006C3074607050F1339DDCB890BB',
  'enaging_user_follower_count': 27428,
  'enaging_user_following_count': 600,
  'enaging_user_is_verified': False,
  'engagee_follows_engager': False,
  'like_timestamp': nan},
 {'tweet_type': 'TopLevel',
  'Language': 'B9175601E87101A984A50F8A62A1C374',
  'tweet_timestamp': 1581018910,
  'enaged_with_user_id': '1F30A85E612A24B426013978ADB7AE46',
  'engaged_with_user_follower_count': 39786,
  'engaged_with_user_following_count': 32978,
  'engaged_with_user_is_verified': False,
  'engaging_user_id': '000005C520010F8917EEAB6F5B6EC1C4',
  'enaging_user_follower_count': 17,
  'enaging_user_following_count': 77,
  'enaging_user_is_verif

In [18]:
>>> from sklearn.feature_extraction import FeatureHasher
>>> h = FeatureHasher(n_features=10)
>>> D = [{'dog': 1, 'cat':2, 'elephant':4},{'dog': 2, 'run': 5}]
>>> f = h.transform(D)
>>> f.toarray()D[0]

dict

In [28]:
train_1.dtypes

tweet_type                            object
Language                              object
tweet_timestamp                        int64
enaged_with_user_id                   object
engaged_with_user_follower_count       int64
engaged_with_user_following_count      int64
engaged_with_user_is_verified           bool
engaging_user_id                      object
enaging_user_follower_count            int64
enaging_user_following_count           int64
enaging_user_is_verified                bool
engagee_follows_engager                 bool
like_timestamp                       float64
dtype: object

In [32]:
h = FeatureHasher(input_type = 'string')

In [21]:
train_subset.head()

,tweet_type,Language,tweet_timestamp,enaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,engagee_follows_engager,like_timestamp
0,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,3517,590,False,0000006C3074607050F1339DDCB890BB,27428,600,False,False,NaN
1,TopLevel,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,39786,32978,False,000005C520010F8917EEAB6F5B6EC1C4,17,77,False,True,NaN
2,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,141726,3232,False,000006829BEADA9EEA695CF0C334B426,2,29,False,False,1.581247e+09
3,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,3044,99,False,000006829BEADA9EEA695CF0C334B426,3,30,False,False,NaN
4,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,False,000006829BEADA9EEA695CF0C334B426,3,30,False,False,NaN


In [33]:
f = h.transform(train_subset['engaging_user_id'])

In [37]:
f

<10x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 128 stored elements in Compressed Sparse Row format>

In [45]:
f.tocoo().col

array([ 22790, 196981, 323322, 500148, 611181, 628086, 669156, 687296,
       689704, 779937, 917481, 930680, 933761, 196981, 323322, 537095,
       611181, 628086, 647703, 669156, 687296, 689704, 779937, 849870,
       917481, 930680, 933761,  22790, 196981, 323322, 500148, 537095,
       628086, 647703, 669156, 687296, 779937, 849870, 917481, 930680,
       933761,  22790, 196981, 323322, 500148, 537095, 628086, 647703,
       669156, 687296, 779937, 849870, 917481, 930680, 933761,  22790,
       196981, 323322, 500148, 537095, 628086, 647703, 669156, 687296,
       779937, 849870, 917481, 930680, 933761,  22790, 196981, 323322,
       500148, 537095, 628086, 647703, 669156, 687296, 779937, 849870,
       917481, 930680, 933761,  22790, 196981, 323322, 500148, 537095,
       611181, 628086, 647703, 669156, 687296, 689704, 779937, 849870,
       917481, 933761,  22790, 196981, 537095, 669156, 687296, 689704,
       779937, 917481, 930680, 933761,  22790, 196981, 537095, 669156,
      

In [7]:
len(train)

148075238

In [5]:
train.dtypes

hashtags                              object
tweet_type                            object
Language                              object
tweet_timestamp                        int64
enaged_with_user_id                   object
engaged_with_user_follower_count       int64
engaged_with_user_following_count      int64
engaged_with_user_is_verified           bool
engaging_user_id                      object
enaging_user_follower_count            int64
enaging_user_following_count           int64
enaging_user_is_verified                bool
engagee_follows_engager                 bool
like_timestamp                       float64
dtype: object

In [3]:
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0

In [4]:
train[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'like_bool']].to_csv('data/like/train.csv', index = False, sep = '\t')

In [5]:
train_set['like_bool'] = train_set.like_timestamp.fillna(0)
train_set.loc[train_set.like_bool != 0.0, 'like_bool'] = 1.0

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [6]:
train_set[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'like_bool']].to_csv('data/like/train_set.csv', index = False, sep = '\t')

In [7]:
val_set['like_bool'] = val_set.like_timestamp.fillna(0)
val_set.loc[val_set.like_bool != 0.0, 'like_bool'] = 1.0

In [8]:
val_set[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'like_bool']].to_csv('data/like/val_set.csv', index = False, sep = '\t')

In [9]:
all_features = ["Language", "tweet_timestamp", "enaged_with_user_id", "engaging_user_id", "engagee_follows_engager",\
                "like_timestamp"]

test = pd.read_csv("s3://recsys-challenge-2020/val.tsv", encoding="utf-8",
                    names = all_features, usecols= [7, 8, 9, 14, 19, 23], sep="\x01")
test['like_bool'] = test.like_timestamp.fillna(0)
test.loc[test.like_bool != 0.0, 'like_bool'] = 1.0
test[['Language', 'enaged_with_user_id', 'engaging_user_id', 'engagee_follows_engager', 'like_bool']].to_csv('data/like/test.csv', index = False, sep = '\t')